<a href="https://colab.research.google.com/github/luiseduaardo/PneuModel/blob/main/notebooks/inferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/luiseduaardo/PneuModel
%cd PneuModel

Cloning into 'PneuModel'...
remote: Enumerating objects: 5902, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 5902 (delta 0), reused 0 (delta 0), pack-reused 5901 (from 3)
Receiving objects: 100% (5902/5902), 1.26 GiB | 14.97 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Updating files: 100% (5863/5863), done.
/content/PneuModel


In [2]:
import tensorflow as tf
import pandas as pd
import os

In [3]:
path_teste = "./data/test"

# carrega o modelo
if os.path.exists('./weights/melhor_modelo_pneumonia.h5'):
  model = tf.keras.models.load_model('./weights/melhor_modelo_pneumonia.h5')
  print("✅ O modelo foi carregado com sucesso")
else:
  print("❌ O modelo não foi carregado")

✅ O modelo foi carregado com sucesso


In [4]:
# pré processamento da imagem
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    path_teste,
    target_size=(384, 384),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# realiza a previsão
previsoes = model.predict(test_generator)

Found 624 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


624/624 ━━━━━━━━━━━━━━━━━━━━ 288s 458ms/step


In [7]:
ids = [os.path.basename(x) for x in test_generator.filenames]

submissao = pd.DataFrame({
    'id': ids,
    'target': previsoes.flatten()
})

submissao.head()

,id,target
0,img_0001.jpeg,0.386153
1,img_0002.jpeg,0.158705
2,img_0003.jpeg,0.998453
3,img_0004.jpeg,0.008347
4,img_0005.jpeg,0.086193


In [8]:
submissao.to_csv('submission.csv', index=False)